In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from utils.plots import plot_acc_loss_keras
from utils.dataloader import load_eeg_data_edf, load_eeg_data_gdf, load_eeg_data_mat
from utils.dataloader import load_eeg_data_mat
from models.transformers import Transformer
import keras
from tensorflow.keras import utils as np_utils
import tensorflow as tf

In [2]:
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [2]:
x_data = np.random.rand(200,64,1000)
y_data = np.random.randint(0,4,200)



In [3]:
_, chans, sample = x_data.shape


X_train, X_aux, Y_train, y_aux = train_test_split( x_data, y_data, test_size=0.5, random_state=42)
X_validate, X_test, Y_validate, Y_test = train_test_split( X_aux, y_aux, test_size=0.5, random_state=42)

Y_train      = np_utils.to_categorical(Y_train)
Y_validate   = np_utils.to_categorical(Y_validate)
Y_test       = np_utils.to_categorical(Y_test)

X_train      = X_train.reshape(-1, chans, sample)
X_validate   = X_validate.reshape(-1, chans, sample)
X_test       = X_test.reshape(-1, chans, sample)

In [5]:
input_shape = X_train.shape[1:]
n_classes = len(np.unique(Y_train))

model = Transformer(input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
    n_classes=n_classes
)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["sparse_categorical_accuracy"],
)
# model.summary()

In [6]:
callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    X_train,
    Y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=64,
    callbacks=callbacks,
)

model.evaluate(X_test, Y_test, verbose=1)

Epoch 1/200


ValueError: in user code:

    File "c:\Users\Maods\Documents\Repos\EEG-Analysis-\env_tcc_eeg\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Maods\Documents\Repos\EEG-Analysis-\env_tcc_eeg\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Maods\Documents\Repos\EEG-Analysis-\env_tcc_eeg\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\Maods\Documents\Repos\EEG-Analysis-\env_tcc_eeg\lib\site-packages\keras\engine\training.py", line 894, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "c:\Users\Maods\Documents\Repos\EEG-Analysis-\env_tcc_eeg\lib\site-packages\keras\engine\training.py", line 987, in compute_metrics
        self.compiled_metrics.update_state(y, 